<a href="https://colab.research.google.com/github/gsgill7/gsgill7/blob/main/Exercises/05%20Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 05.1 (random numbers)

- Using the `randint` function from the `random` module (https://docs.python.org/3/library/random.html#random.randint) to
  develop a function `dice_roll` that emulates the roll of a dice with $n$ sides. The number of sides `n` should an argument to the function.

- For $n=6$, devise and implement a test to check that it is a fair dice.

#### (a) Dice roll code:

In [6]:
import random
def dice_roll(n):
  roll = random.randint(1,n)
  return roll

In [7]:
## tests ##
for n in range(1, 20):
    for j in range(100):
        value = dice_roll(n)
        assert value >= 1 and value <= n

#### (b) Test for fairness

In [12]:
# To check for fairness, we roll the dice a large number of times and check how many times we get each value.
# We expect to get each value close to $1/6$ of the times.
roll_values=[0,0,0,0,0,0]
n_rolls = 1200000
for n in range (0,n_rolls):
  roll = dice_roll(6)
  roll_values[roll-1] += 1
print(roll_values)
for value in roll_values:
  print(roll_values.index(value)+1 , (1-(value/(n_rolls//6)))*100)

[200393, 199490, 200220, 200071, 200049, 199777]
1 -0.19649999999999945
2 0.2550000000000052
3 -0.11000000000001009
4 -0.03550000000001052
5 -0.024500000000005073
6 0.11149999999999771


## Exercise 05.2 (variance estimation)

For a random variable $X$, the variance of $X$ is defined as  

$$
\begin{align}
\mathrm{Var}\left[ X \right] &= \mathrm{E}\left[\left( X - \mu \right)^2\right]  \\
&= \mathrm{E}\left[ X^{2} \right] - \mathrm{E}\left[ X \right]^2
\end{align}
$$

where $\mathrm{E}$ is the 'expectation' (mean of something) and $\mu = \mathrm{E}(X)$ is the mean of $X$. If we have all data (the entire 'population'),
the variance can be computed from:

$$
\mathrm{Var}\left[ X \right] = \frac{\sum_{i=0}^{N-1} x^{2}_{i}}{N} - \left( \frac{ \left( \sum_{i=0}^{N-1} x_{i} \right)}{N} \right)^{2}
$$

Often, we only have a sample of data. For example, we might want to estimate the variance in height for students at a university using just a random sample of students.
When using a sample from a larger data set to estimate the variance, the above formula has a *bias*. Therefore, it is common to use the *unbiased* estimator

$$
s^{2} = \left( \frac{\sum_{i=0}^{n-1} x^{2}_{i}}{n} - \left( \frac{ \left( \sum_{i=0}^{n-1} x_{i} \right)}{n} \right)^{2} \right) \frac{n}{n-1}
$$

to estimate the variance. In this exercise we will use the unbiased estimator.

1. Create a function that returns the estimated variance for a list of numbers based on the above equation. Test your function using 1 million samples drawn from a Gaussian distribution with a mean of 10 and a standard deviation of 3. Use your crsid to seed the random number generator
   
   *See hint below on how to create the sample.*

2. For a sample drawn from a distribution with mean $5 \times 10^6$ and standard deviation $2.0$, estimate the variance using (i) your function for estimating the variance, and (ii) using the `variance` function from the Python `statistics` module. Comment on and explain any significant differences in the results from (i) and (ii).

### Hint: sampling from a distribution

The function `random.gauss` can be used to sample a Gaussian distribution with a specified mean and standard deviation (square root of the variance) *N* times, e.g.:

In [13]:
import random

random.seed("gsg27")  # See the random number generator

mu = 10.0    # mean of the distribution
sigma = 3.0  # Standard deviation
x = [random.gauss(mu, sigma) for i in range(8)]
print(x)

[16.11054500918577, 6.197176283209556, 6.6685586430367, 12.860061528122342, 16.400338694586356, 7.141555964766771, 7.12018173076307, 4.215875907517725]


### Solution

1. Estimate variance

In [14]:
import random

mu = 10.0    # mean of the distribution
sigma = 3.0  # Standard deviation
x = [random.gauss(mu, sigma) for i in range(8)]
print(x)


random.seed("gsg27")
def e(x):
  sum = 0
  for i in range(len(x)):
    sum += x[i]
  return sum/len(x)
def e_squared(x):
  sum = 0
  for i in range(len(x)):
    sum += x[i]**2
  return sum/len(x)
def variance(x):
  variance = (e_squared(x)-e(x)**2)*(len(x)/(len(x)-1))
  return variance
print(variance(x))

[9.698397054582696, 14.838962120856172, 12.374966989118485, 4.599557323097943, 10.387122695416199, 11.068491466324526, 9.620258486003637, 14.215059687424684]
10.247064277240561


2. Use the `statistics` module to estimate the variance, and compare the estimated variance using `statistics` to the estimated variance using your implementation for the variance estimation.

In [17]:
import statistics
print(statistics.variance(x))
print(variance(x))
print(((statistics.variance(x)-variance(x))/statistics.variance(x))*100)

10.247064277240534
10.247064277240561
-2.600291348829049e-13


## Exercise 05.03 (optional, parallel processing)

Almost all modern computer processing units have multiple processing *cores*. To utilise the full performance of a processor, operations need to be performed in *parallel*, i.e. each processing core is given a task to perform.

Parallel computing is a very rich and technical area. To help exploit multi-core systems there are libraries that support parallel processing. Below is a simple example using the Python `multiprocessing` library.

In [ ]:
import multiprocessing

# On some operaring systems, you may need to uncomment the below line
# multiprocessing.set_start_method('fork')

def f(task):
    """A function that print the input argument and the id for the process that executes the function"""
    print(f"Task index {task}, process id: {multiprocessing.current_process()}\n")

    return str(multiprocessing.current_process())


# Using 3 'processes', execute the function 'f' four times (each time with a different argument)
with multiprocessing.Pool(processes=3) as p:
    procs = p.map(f, [0, 1, 2, 3])  # Call function mysort three times

    print("Returned data (a list)")
    print(procs)

Consider the below code that creates a list of lists of sorted numbers.

In [ ]:
def mysort(N):
    """Create a randomly ordered list of integers of length N, and return the sorted list"""
    # Create randomly ordered list
    x = random.sample(range(0, N), N)

    # Return sorted list of numbers
    return sorted(x)

def sorted_lists(N, p):
    """Create a list of sorted lists"""
    data = []
    for i in range(p):
        data.append(mysort(N))

%time x = sorted_lists(1000000, 5)

Use the `multiprocessing` module to perform the above operation in parallel. Investigate how the processing time changes with the number of processes, and in particular the average time per 'create and sort operation' when changing the number of processes.

In [ ]:
...